In [ ]:
import os
# import pkg_resources
# pkg_resources.require('Torch==2.0.1')
!pip install torch==2.0.1
import torch
print('Using torch', torch.__version__)
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install ogb
!pip install rdkit-pypi


from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import global_mean_pool
from torch.autograd import Variable

import matplotlib
import torch_geometric

import math
import numpy as np
import matplotlib.pyplot as plt
import os.path as osp
import time

import networkx as nx
import random


from torch_geometric.nn.inits import uniform

from torch_scatter import scatter_mean

from torch_geometric.datasets import TUDataset
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.nn import GCNConv, GraphMultisetTransformer

from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GCNConv, GINConv
from torch_geometric.utils import degree
from torch_geometric.utils import to_dense_batch


from ogb.graphproppred.dataset_pyg import PygGraphPropPredDataset
from ogb.graphproppred import Evaluator

from ogb.graphproppred.mol_encoder import BondEncoder

import torch.nn as nn
import torch.nn.functional as F

from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys
import csv
import re

from torch.nn import Linear
from tqdm import tqdm

from argparse import Namespace
from torch_geometric.nn import global_mean_pool, global_max_pool as gmp, global_add_pool as gsp
from torch_geometric.utils import to_dense_batch

from ogb.graphproppred.mol_encoder import AtomEncoder

from math import ceil

from logging import Logger
from torch.optim.lr_scheduler import ExponentialLR
import torch.optim as optim
from torch_geometric.data import Batch


# set seed
seed_value = 37
torch.manual_seed(seed_value)

class MAB(nn.Module):
    def __init__(self, dim_Q, dim_K, dim_V, num_heads, ln=False, cluster=False, conv=None):
        super(MAB, self).__init__()
        self.dim_V = dim_V
        self.num_heads = num_heads
        self.fc_q = nn.Linear(dim_Q, dim_V)

        self.fc_k, self.fc_v = self.get_fc_kv(dim_K, dim_V, conv)

        if ln:
            self.ln0 = nn.LayerNorm(dim_V)
            self.ln1 = nn.LayerNorm(dim_V)
        self.fc_o = nn.Linear(dim_V, dim_V)

        self.softmax_dim = 2
        if cluster == True:
            self.softmax_dim = 1

    def forward(self, Q, K, attention_mask=None, graph=None, return_attn=False):
        Q = self.fc_q(Q)

        # Adj: Exist (graph is not None), or Identity (else)
        if graph is not None:

            (x, edge_index, batch) = graph

            K, V = self.fc_k(x, edge_index), self.fc_v(x, edge_index)

            K, _ = to_dense_batch(K, batch)
            V, _ = to_dense_batch(V, batch)

        else:

            K, V = self.fc_k(K), self.fc_v(K)

        dim_split = self.dim_V // self.num_heads
        Q_ = torch.cat(Q.split(dim_split, 2), 0)
        K_ = torch.cat(K.split(dim_split, 2), 0)
        V_ = torch.cat(V.split(dim_split, 2), 0)

        if attention_mask is not None:
            attention_mask = torch.cat([attention_mask for _ in range(self.num_heads)], 0)
            attention_score = Q_.bmm(K_.transpose(1,2))/math.sqrt(self.dim_V)
            A = torch.softmax(attention_mask + attention_score, self.softmax_dim)
        else:
            A = torch.softmax(Q_.bmm(K_.transpose(1,2))/math.sqrt(self.dim_V), self.softmax_dim)

        O = torch.cat((Q_ + A.bmm(V_)).split(Q.size(0), 0), 2)
        O = O if getattr(self, 'ln0', None) is None else self.ln0(O)
        O = O + F.relu(self.fc_o(O))
        O = O if getattr(self, 'ln1', None) is None else self.ln1(O)
        if return_attn:
            return O, A
        else:
            return O

    def get_fc_kv(self, dim_K, dim_V, conv):

        if conv == 'GCN':

            fc_k = GCNConv(dim_K, dim_V)
            fc_v = GCNConv(dim_K, dim_V)

        elif conv == 'GIN':

            fc_k = GINConv(
                nn.Sequential(
                    nn.Linear(dim_K, dim_K),
                    nn.ReLU(),
                    nn.Linear(dim_K, dim_V),
                    nn.ReLU(),
                    nn.BatchNorm1d(dim_V),
            ), train_eps=False)

            fc_v = GINConv(
                nn.Sequential(
                    nn.Linear(dim_K, dim_K),
                    nn.ReLU(),
                    nn.Linear(dim_K, dim_V),
                    nn.ReLU(),
                    nn.BatchNorm1d(dim_V),
            ), train_eps=False)

        else:

            fc_k = nn.Linear(dim_K, dim_V)
            fc_v = nn.Linear(dim_K, dim_V)

        return fc_k, fc_v

class SAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, ln=False, cluster=False, mab_conv=None):
        super(SAB, self).__init__()

        self.mab = MAB(dim_in, dim_in, dim_out, num_heads, ln=ln, cluster=cluster, conv=mab_conv)

    def forward(self, X, attention_mask=None, graph=None):
        return self.mab(X, X, attention_mask, graph)

class ISAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, num_inds, ln=False, cluster=False, mab_conv=None):
        super(ISAB, self).__init__()
        self.I = nn.Parameter(torch.Tensor(1, num_inds, dim_out))
        nn.init.xavier_uniform_(self.I)

        self.mab0 = MAB(dim_out, dim_in, dim_out, num_heads, ln=ln, cluster=cluster, conv=mab_conv)
        self.mab1 = MAB(dim_in, dim_out, dim_out, num_heads, ln=ln, cluster=cluster, conv=mab_conv)

    def forward(self, X, attention_mask=None, graph=None):
        H = self.mab0(self.I.repeat(X.size(0), 1, 1), X, attention_mask, graph)
        return self.mab1(X, H)

class PMA(nn.Module):
    def __init__(self, dim, num_heads, num_seeds, ln=False, cluster=False, mab_conv=None):
        super(PMA, self).__init__()
        self.S = nn.Parameter(torch.Tensor(1, num_seeds, dim))
        nn.init.xavier_uniform_(self.S)

        self.mab = MAB(dim, dim, dim, num_heads, ln=ln, cluster=cluster, conv=mab_conv)

    def forward(self, X, attention_mask=None, graph=None, return_attn=False):
        return self.mab(self.S.repeat(X.size(0), 1, 1), X, attention_mask, graph, return_attn)

### GCN convolution along the graph structure
class GCNConv_for_OGB(MessagePassing):
    def __init__(self, emb_dim):
        super(GCNConv_for_OGB, self).__init__(aggr='add')

        self.linear = torch.nn.Linear(emb_dim, emb_dim)
        self.root_emb = torch.nn.Embedding(1, emb_dim)
        self.bond_encoder = BondEncoder(emb_dim = emb_dim)

    def forward(self, x, edge_index, edge_attr):
        x = self.linear(x)
        edge_embedding = self.bond_encoder(edge_attr)

        row, col = edge_index

        deg = degree(row, x.size(0), dtype = x.dtype) + 1
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return self.propagate(edge_index, x=x, edge_attr=edge_embedding, norm=norm) + F.relu(x + self.root_emb.weight) * 1./deg.view(-1,1)

    def message(self, x_j, edge_attr, norm):
        return norm.view(-1, 1) * F.relu(x_j + edge_attr)

    def update(self, aggr_out):
        return aggr_out

### GIN convolution along the graph structure
class GINConv_for_OGB(MessagePassing):
    def __init__(self, emb_dim):
        super(GINConv_for_OGB, self).__init__(aggr = "add")

        self.mlp = torch.nn.Sequential(torch.nn.Linear(emb_dim, 2*emb_dim), torch.nn.BatchNorm1d(2*emb_dim), torch.nn.ReLU(), torch.nn.Linear(2*emb_dim, emb_dim))
        self.eps = torch.nn.Parameter(torch.Tensor([0]))

        self.bond_encoder = BondEncoder(emb_dim = emb_dim)

    def forward(self, x, edge_index, edge_attr):
        edge_embedding = self.bond_encoder(edge_attr)
        out = self.mlp((1 + self.eps) * x + self.propagate(edge_index, x=x, edge_attr=edge_embedding))

        return out

    def message(self, x_j, edge_attr):
        return F.relu(x_j + edge_attr)

    def update(self, aggr_out):
        return aggr_out

class GraphRepresentation(torch.nn.Module):

    def __init__(self, args):

        super(GraphRepresentation, self).__init__()

        self.args = args
        self.num_features = args.num_features
        self.nhid = args.num_hidden
        self.num_classes = args.num_classes
        self.pooling_ratio = args.pooling_ratio
        self.dropout_ratio = args.dropout

    def get_convs(self):

        convs = nn.ModuleList()

        _input_dim = self.num_features
        _output_dim = self.nhid

        for _ in range(self.args.num_convs):

            if self.args.conv == 'GCN':

                conv = GCNConv(_input_dim, _output_dim)

            elif self.args.conv == 'GIN':

                conv = GINConv(
                    nn.Sequential(
                        nn.Linear(_input_dim, _output_dim),
                        nn.ReLU(),
                        nn.Linear(_output_dim, _output_dim),
                        nn.ReLU(),
                        nn.BatchNorm1d(_output_dim),
                ), train_eps=False)

            convs.append(conv)

            _input_dim = _output_dim
            _output_dim = _output_dim

        return convs

    def get_pools(self):

        pools = nn.ModuleList([gap])

        return pools

    def get_classifier(self):

        return nn.Sequential(
            nn.Linear(self.nhid, self.nhid),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_ratio),
            nn.Linear(self.nhid, self.nhid//2),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_ratio),
            nn.Linear(self.nhid//2, self.num_classes)
        )

class GraphMultisetTransformer(GraphRepresentation):

    def __init__(self, args):

        super(GraphMultisetTransformer, self).__init__(args)

        self.ln = args.ln
        self.num_heads = args.num_heads
        self.cluster = args.cluster

        self.model_sequence = args.model_string.split('-')

        self.convs = self.get_convs()
        self.pools = self.get_pools()
        self.classifier = self.get_classifier()

    def forward(self, data):

        x, edge_index, batch = data.x, data.edge_index, data.batch

        # For Graph Convolution Network
        xs = []

        for _ in range(self.args.num_convs):

            x = F.relu(self.convs[_](x, edge_index))
            xs.append(x)

        # For jumping knowledge scheme
        x = torch.cat(xs, dim=1)

        # For Graph Multiset Transformer
        for _index, _model_str in enumerate(self.model_sequence):

            if _index == 0:

                batch_x, mask = to_dense_batch(x, batch)

                extended_attention_mask = mask.unsqueeze(1)
                extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)
                extended_attention_mask = (1.0 - extended_attention_mask) * -1e9

            if _model_str == 'GMPool_G':

                batch_x = self.pools[_index](batch_x, attention_mask=extended_attention_mask, graph=(x, edge_index, batch))

            else:

                batch_x = self.pools[_index](batch_x, attention_mask=extended_attention_mask)

            extended_attention_mask = None

        batch_x = self.pools[len(self.model_sequence)](batch_x)
        x = batch_x.squeeze(1)

        # For Classification
        x = self.classifier(x)

        return F.log_softmax(x, dim=-1)

    def get_pools(self, _input_dim=None, reconstruction=False):

        pools = nn.ModuleList()

        _input_dim = self.nhid * self.args.num_convs if _input_dim is None else _input_dim
        _output_dim = self.nhid
        _num_nodes = ceil(self.pooling_ratio * self.args.avg_num_nodes)

        for _index, _model_str in enumerate(self.model_sequence):

            if (_index == len(self.model_sequence) - 1) and (reconstruction == False):

                _num_nodes = 1

            if _model_str == 'GMPool_G':

                pools.append(
                    PMA(_input_dim, self.num_heads, _num_nodes, ln=self.ln, cluster=self.cluster, mab_conv=self.args.mab_conv)
                )

                _num_nodes = ceil(self.pooling_ratio * _num_nodes)

            elif _model_str == 'GMPool_I':

                pools.append(
                    PMA(_input_dim, self.num_heads, _num_nodes, ln=self.ln, cluster=self.cluster, mab_conv=None)
                )

                _num_nodes = ceil(self.pooling_ratio * _num_nodes)

            elif _model_str == 'SelfAtt':

                pools.append(
                    SAB(_input_dim, _output_dim, self.num_heads, ln=self.ln, cluster=self.cluster)
                )

                _input_dim = _output_dim
                _output_dim = _output_dim

            else:

                raise ValueError("Model Name in Model String <{}> is Unknown".format(_model_str))

        pools.append(nn.Linear(_input_dim, self.nhid))

        return pools

class GraphMultisetTransformer_for_OGB(GraphMultisetTransformer):

    def __init__(self, args):

        super(GraphMultisetTransformer_for_OGB, self).__init__(args)

        self.atom_encoder = AtomEncoder(self.nhid)
        self.convs = self.get_convs()

    def forward(self, data):

        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

        x = self.atom_encoder(x)

        # For Graph Convolution Network
        xs = []

        for _ in range(self.args.num_convs):

            x = F.relu(self.convs[_](x, edge_index, edge_attr))
            xs.append(x)

        # For jumping knowledge scheme
        x = torch.cat(xs, dim=1)

        # For Graph Multiset Transformer
        for _index, _model_str in enumerate(self.model_sequence):

            if _index == 0:

                batch_x, mask = to_dense_batch(x, batch)

                extended_attention_mask = mask.unsqueeze(1)
                extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)
                extended_attention_mask = (1.0 - extended_attention_mask) * -1e9

            if _model_str == 'GMPool_G':

                batch_x = self.pools[_index](batch_x, attention_mask=extended_attention_mask, graph=(x, edge_index, batch))

            else:

                batch_x = self.pools[_index](batch_x, attention_mask=extended_attention_mask)

            extended_attention_mask = None

        batch_x = self.pools[len(self.model_sequence)](batch_x)
        x = batch_x.squeeze(1)

        # For Classification
        x = self.classifier(x)

        return x

    def get_convs(self):

        convs = nn.ModuleList()

        for _ in range(self.args.num_convs):

            if self.args.conv == 'GCN':

                conv = GCNConv_for_OGB(self.nhid)

            elif self.args.conv == 'GIN':

                conv = GINConv_for_OGB(self.nhid)

            convs.append(conv)

        return convs

def plot_training_metrics(losses, val_metrics, test_metrics, epoch_times):
    """
    Plot training loss, validation metric, and test metric over epochs.

    Parameters:
    - losses: List of training losses for each epoch.
    - val_metrics: List of validation metrics for each epoch.
    - test_metrics: List of test metrics for each epoch.
    """
    epochs = range(1, len(losses) + 1)

    # Plotting training loss
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 3, 1)
    plt.plot(epochs, losses, label='Training Loss', marker='o')
    plt.title('Training Loss vs. Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plotting validation and test metrics
    plt.subplot(1, 3, 2)
    plt.plot(epochs, val_metrics, label='Validation Metric', marker='o')
    plt.plot(epochs, test_metrics, label='Test Metric', marker='o')
    plt.title('Validation and Test Metrics vs. Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Metric')
    plt.legend()

    # Plotting time vs. epoch
    plt.subplot(1, 3, 3)
    plt.plot(epochs, epoch_times, label='Time per Epoch', marker='o', color='orange')
    plt.title('Time vs. Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Time (seconds)')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Example usage:
# plot_training_metrics(training_losses, validation_metrics, test_metrics)



def tud_trans(dataset_name, k, h):
  path = 'tmp\tud'
  dataset = TUDataset(path, name=dataset_name).shuffle()

  n = (len(dataset) + 9) // 10
  train_dataset = dataset[2 * n:]
  val_dataset = dataset[n:2 * n]
  test_dataset = dataset[:n]

  train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=128)
  test_loader = DataLoader(test_dataset, batch_size=128)


  class Net(torch.nn.Module):
      def __init__(self):
          super().__init__()

          self.conv1 = GCNConv(dataset.num_features, 32)
          self.conv2 = GCNConv(32, 32)
          self.conv3 = GCNConv(32, 32)

          self.pool = GraphMultisetTransformer(96, k=k, heads=h)

          self.lin1 = Linear(96, 16)
          self.lin2 = Linear(16, dataset.num_classes)

      def forward(self, x0, edge_index, batch):
          x1 = self.conv1(x0, edge_index).relu()
          x2 = self.conv2(x1, edge_index).relu()
          x3 = self.conv3(x2, edge_index).relu()
          x = torch.cat([x1, x2, x3], dim=-1)

          x = self.pool(x, batch)

          x = self.lin1(x).relu()
          x = F.dropout(x, p=0.5, training=self.training)
          x = self.lin2(x)

          return x


  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = Net().to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)


  def train():
      model.train()

      total_loss = 0
      for data in train_loader:
          data = data.to(device)
          optimizer.zero_grad()
          out = model(data.x, data.edge_index, data.batch)
          loss = F.cross_entropy(out, data.y)
          loss.backward()
          total_loss += data.num_graphs * float(loss)
          optimizer.step()
      return total_loss / len(train_dataset)


  @torch.no_grad()
  def test(loader):
      model.eval()

      total_correct = 0
      for data in loader:
          data = data.to(device)
          out = model(data.x, data.edge_index, data.batch)
          total_correct += int((out.argmax(dim=-1) == data.y).sum())
      return total_correct / len(loader.dataset)


  times = []
  training_losses = []
  validation_metrics = []
  test_metrics = []
  for epoch in range(1, 101):
      start = time.time()
      train_loss = train()
      val_acc = test(val_loader)
      test_acc = test(test_loader)
      training_losses.append(train_loss)
      validation_metrics.append(val_acc)
      test_metrics.append(test_acc)
      print(f'Epoch: {epoch:03d}, Loss: {train_loss:.4f}, '
            f'Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
      times.append(time.time() - start)
  print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

  plot_training_metrics(training_losses, validation_metrics, test_metrics, times)
  print(f'Best Validation Metric: {max(validation_metrics)}')
  print(f'Test Metric: {test_metrics[np.argmax(validation_metrics)]}')

tud_trans('PROTEINS', k=10, h=4)
tud_trans('DD', k=8, h=2)

# baseline GCNConv comparison
def conv_TUD(dataset_name):

  path = '\tmp\tud'
  dataset = TUDataset(path, dataset_name).shuffle()

  num_classes = dataset.num_classes
  num_features = dataset.num_features
  hidden_channels = 32
  task_type='classification'
  # print(num_classes)

  def collate(data_list):
    batch = torch_geometric.data.Batch.from_data_list(data_list)
    return batch

  total_size = len(dataset)
  train_size = int(0.8 * total_size)
  test_size = (total_size - train_size) // 2
  val_size = total_size - train_size - test_size

  # Use random_split to split the dataset into train, test, and validation sets
  train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])

  train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=collate)
  val_loader = DataLoader(val_dataset, batch_size=128, collate_fn=collate)
  test_loader = DataLoader(test_dataset, batch_size=128, collate_fn=collate)

  class NN(torch.nn.Module):
      def __init__(self):
          super().__init__()

          self.conv1 = GCNConv(in_channels=num_features, out_channels=hidden_channels, normalize=True)
          self.conv2 = GCNConv(in_channels=hidden_channels, out_channels=num_classes, normalize=True)
          self.relu = torch.nn.ReLU()

          self.lin1 = Linear(32, 16)
          self.lin2 = Linear(16, num_classes)

      def forward(self, x0, edge_index, batch):
          x = self.conv1(x0, edge_index)
          x = self.relu(x)
          x = self.conv2(x, edge_index)
          # global mean pool instead of GMT, along with other adjustments
          output = global_mean_pool(x, batch)
          # output = output.detach().clone().requires_grad_(True)
          # x = self.lin1(x)
          # output = self.lin2(x)

          return output


  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = NN().to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

  def train(loader):
      model.train()

      total_loss = 0

      epoch_iter = tqdm(loader, ncols=130)
      for step, batch in enumerate(epoch_iter):
          batch = batch.to(device)
          optimizer.zero_grad()
          out = model(batch.x, batch.edge_index, batch.batch)
          is_labeled = ~torch.isnan(batch.y) # Mask NaNs
          loss = F.cross_entropy(out[is_labeled], batch.y[is_labeled])
          loss.backward()
          # prevent exploding gradients if needed, uncomment
          # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
          total_loss += batch.num_graphs * float(loss)
          optimizer.step()
      return total_loss / len(train_dataset)


  @torch.no_grad()
  def test(loader):

      model.eval()

      total_same = 0
      epoch_iter = tqdm(loader, ncols=130)
      for step, batch in enumerate(epoch_iter):
          batch = batch.to(device)
          out = model(batch.x, batch.edge_index, batch.batch)
          total_same += int((out.argmax(dim=-1) == batch.y).sum())
      return total_same / len(loader.dataset)


  times = []
  training_losses = []
  validation_metrics = []
  test_metrics = []
  for epoch in range(1, 201):
      start = time.time()
      train_loss = train(train_loader)
      val_acc = test(val_loader)
      test_acc = test(test_loader)
      training_losses.append(train_loss)
      validation_metrics.append(val_acc)
      test_metrics.append(test_acc)
      print(f'Epoch: {epoch:03d}, Loss: {train_loss:.4f}, '
            f'Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
      times.append(time.time() - start)
  print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")
  plot_training_metrics(training_losses, validation_metrics, test_metrics, times)
  print(f'Best Validation Metric: {max(validation_metrics)}')
  print(f'Test Metric: {test_metrics[np.argmax(validation_metrics)]}')

conv_TUD('DD')
conv_TUD('PROTEINS')

# set seed
seed_value = 108
torch.manual_seed(seed_value)

class MAB(nn.Module):
    def __init__(self, dim_Q, dim_K, dim_V, num_heads, ln=False, cluster=False, conv=None):
        super(MAB, self).__init__()
        self.dim_V = dim_V
        self.num_heads = num_heads
        self.fc_q = nn.Linear(dim_Q, dim_V)

        self.fc_k, self.fc_v = self.get_fc_kv(dim_K, dim_V, conv)

        if ln:
            self.ln0 = nn.LayerNorm(dim_V)
            self.ln1 = nn.LayerNorm(dim_V)
        self.fc_o = nn.Linear(dim_V, dim_V)

        self.softmax_dim = 2
        if cluster == True:
            self.softmax_dim = 1

    def forward(self, Q, K, attention_mask=None, graph=None, return_attn=False):
        Q = self.fc_q(Q)

        # Adj: Exist (graph is not None), or Identity (else)
        if graph is not None:

            (x, edge_index, batch) = graph

            K, V = self.fc_k(x, edge_index), self.fc_v(x, edge_index)

            K, _ = to_dense_batch(K, batch)
            V, _ = to_dense_batch(V, batch)

        else:

            K, V = self.fc_k(K), self.fc_v(K)

        dim_split = self.dim_V // self.num_heads
        Q_ = torch.cat(Q.split(dim_split, 2), 0)
        K_ = torch.cat(K.split(dim_split, 2), 0)
        V_ = torch.cat(V.split(dim_split, 2), 0)

        if attention_mask is not None:
            attention_mask = torch.cat([attention_mask for _ in range(self.num_heads)], 0)
            attention_score = Q_.bmm(K_.transpose(1,2))/math.sqrt(self.dim_V)
            A = torch.softmax(attention_mask + attention_score, self.softmax_dim)
        else:
            A = torch.softmax(Q_.bmm(K_.transpose(1,2))/math.sqrt(self.dim_V), self.softmax_dim)

        O = torch.cat((Q_ + A.bmm(V_)).split(Q.size(0), 0), 2)
        O = O if getattr(self, 'ln0', None) is None else self.ln0(O)
        O = O + F.relu(self.fc_o(O))
        O = O if getattr(self, 'ln1', None) is None else self.ln1(O)
        if return_attn:
            return O, A
        else:
            return O

    def get_fc_kv(self, dim_K, dim_V, conv):

        if conv == 'GCN':

            fc_k = GCNConv(dim_K, dim_V)
            fc_v = GCNConv(dim_K, dim_V)

        elif conv == 'GIN':

            fc_k = GINConv(
                nn.Sequential(
                    nn.Linear(dim_K, dim_K),
                    nn.ReLU(),
                    nn.Linear(dim_K, dim_V),
                    nn.ReLU(),
                    nn.BatchNorm1d(dim_V),
            ), train_eps=False)

            fc_v = GINConv(
                nn.Sequential(
                    nn.Linear(dim_K, dim_K),
                    nn.ReLU(),
                    nn.Linear(dim_K, dim_V),
                    nn.ReLU(),
                    nn.BatchNorm1d(dim_V),
            ), train_eps=False)

        else:

            fc_k = nn.Linear(dim_K, dim_V)
            fc_v = nn.Linear(dim_K, dim_V)

        return fc_k, fc_v

class SAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, ln=False, cluster=False, mab_conv=None):
        super(SAB, self).__init__()

        self.mab = MAB(dim_in, dim_in, dim_out, num_heads, ln=ln, cluster=cluster, conv=mab_conv)

    def forward(self, X, attention_mask=None, graph=None):
        return self.mab(X, X, attention_mask, graph)

class ISAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, num_inds, ln=False, cluster=False, mab_conv=None):
        super(ISAB, self).__init__()
        self.I = nn.Parameter(torch.Tensor(1, num_inds, dim_out))
        nn.init.xavier_uniform_(self.I)

        self.mab0 = MAB(dim_out, dim_in, dim_out, num_heads, ln=ln, cluster=cluster, conv=mab_conv)
        self.mab1 = MAB(dim_in, dim_out, dim_out, num_heads, ln=ln, cluster=cluster, conv=mab_conv)

    def forward(self, X, attention_mask=None, graph=None):
        H = self.mab0(self.I.repeat(X.size(0), 1, 1), X, attention_mask, graph)
        return self.mab1(X, H)

class PMA(nn.Module):
    def __init__(self, dim, num_heads, num_seeds, ln=False, cluster=False, mab_conv=None):
        super(PMA, self).__init__()
        self.S = nn.Parameter(torch.Tensor(1, num_seeds, dim))
        nn.init.xavier_uniform_(self.S)

        self.mab = MAB(dim, dim, dim, num_heads, ln=ln, cluster=cluster, conv=mab_conv)

    def forward(self, X, attention_mask=None, graph=None, return_attn=False):
        return self.mab(self.S.repeat(X.size(0), 1, 1), X, attention_mask, graph, return_attn)

### GCN convolution along the graph structure
class GCNConv_for_OGB(MessagePassing):
    def __init__(self, emb_dim):
        super(GCNConv_for_OGB, self).__init__(aggr='add')

        self.linear = torch.nn.Linear(emb_dim, emb_dim)
        self.root_emb = torch.nn.Embedding(1, emb_dim)
        self.bond_encoder = BondEncoder(emb_dim = emb_dim)

    def forward(self, x, edge_index, edge_attr):
        x = self.linear(x)
        edge_embedding = self.bond_encoder(edge_attr)

        row, col = edge_index

        deg = degree(row, x.size(0), dtype = x.dtype) + 1
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return self.propagate(edge_index, x=x, edge_attr=edge_embedding, norm=norm) + F.relu(x + self.root_emb.weight) * 1./deg.view(-1,1)

    def message(self, x_j, edge_attr, norm):
        return norm.view(-1, 1) * F.relu(x_j + edge_attr)

    def update(self, aggr_out):
        return aggr_out

### GIN convolution along the graph structure
class GINConv_for_OGB(MessagePassing):
    def __init__(self, emb_dim):
        super(GINConv_for_OGB, self).__init__(aggr = "add")

        self.mlp = torch.nn.Sequential(torch.nn.Linear(emb_dim, 2*emb_dim), torch.nn.BatchNorm1d(2*emb_dim), torch.nn.ReLU(), torch.nn.Linear(2*emb_dim, emb_dim))
        self.eps = torch.nn.Parameter(torch.Tensor([0]))

        self.bond_encoder = BondEncoder(emb_dim = emb_dim)

    def forward(self, x, edge_index, edge_attr):
        edge_embedding = self.bond_encoder(edge_attr)
        out = self.mlp((1 + self.eps) * x + self.propagate(edge_index, x=x, edge_attr=edge_embedding))

        return out

    def message(self, x_j, edge_attr):
        return F.relu(x_j + edge_attr)

    def update(self, aggr_out):
        return aggr_out

class GraphRepresentation(torch.nn.Module):

    def __init__(self, args):

        super(GraphRepresentation, self).__init__()

        self.args = args
        self.num_features = args.num_features
        self.nhid = args.num_hidden
        self.num_classes = args.num_classes
        self.pooling_ratio = args.pooling_ratio
        self.dropout_ratio = args.dropout

    def get_convs(self):

        convs = nn.ModuleList()

        _input_dim = self.num_features
        _output_dim = self.nhid

        for _ in range(self.args.num_convs):

            if self.args.conv == 'GCN':

                conv = GCNConv(_input_dim, _output_dim)

            elif self.args.conv == 'GIN':

                conv = GINConv(
                    nn.Sequential(
                        nn.Linear(_input_dim, _output_dim),
                        nn.ReLU(),
                        nn.Linear(_output_dim, _output_dim),
                        nn.ReLU(),
                        nn.BatchNorm1d(_output_dim),
                ), train_eps=False)

            convs.append(conv)

            _input_dim = _output_dim
            _output_dim = _output_dim

        return convs

    def get_pools(self):

        pools = nn.ModuleList([gap])

        return pools

    def get_classifier(self):

        return nn.Sequential(
            nn.Linear(self.nhid, self.nhid),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_ratio),
            nn.Linear(self.nhid, self.nhid//2),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_ratio),
            nn.Linear(self.nhid//2, self.num_classes)
        )

class GraphMultisetTransformer(GraphRepresentation):

    def __init__(self, args):

        super(GraphMultisetTransformer, self).__init__(args)

        self.ln = args.ln
        self.num_heads = args.num_heads
        self.cluster = args.cluster

        self.model_sequence = args.model_string.split('-')

        self.convs = self.get_convs()
        self.pools = self.get_pools()
        self.classifier = self.get_classifier()

    def forward(self, data):

        x, edge_index, batch = data.x, data.edge_index, data.batch

        # For Graph Convolution Network
        xs = []

        for _ in range(self.args.num_convs):

            x = F.relu(self.convs[_](x, edge_index))
            xs.append(x)

        # For jumping knowledge scheme
        x = torch.cat(xs, dim=1)

        # For Graph Multiset Transformer
        for _index, _model_str in enumerate(self.model_sequence):

            if _index == 0:

                batch_x, mask = to_dense_batch(x, batch)

                extended_attention_mask = mask.unsqueeze(1)
                extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)
                extended_attention_mask = (1.0 - extended_attention_mask) * -1e9

            if _model_str == 'GMPool_G':

                batch_x = self.pools[_index](batch_x, attention_mask=extended_attention_mask, graph=(x, edge_index, batch))

            else:

                batch_x = self.pools[_index](batch_x, attention_mask=extended_attention_mask)

            extended_attention_mask = None

        batch_x = self.pools[len(self.model_sequence)](batch_x)
        x = batch_x.squeeze(1)

        # For Classification
        x = self.classifier(x)

        return F.log_softmax(x, dim=-1)

    def get_pools(self, _input_dim=None, reconstruction=False):

        pools = nn.ModuleList()

        _input_dim = self.nhid * self.args.num_convs if _input_dim is None else _input_dim
        _output_dim = self.nhid
        _num_nodes = ceil(self.pooling_ratio * self.args.avg_num_nodes)

        for _index, _model_str in enumerate(self.model_sequence):

            if (_index == len(self.model_sequence) - 1) and (reconstruction == False):

                _num_nodes = 1

            if _model_str == 'GMPool_G':

                pools.append(
                    PMA(_input_dim, self.num_heads, _num_nodes, ln=self.ln, cluster=self.cluster, mab_conv=self.args.mab_conv)
                )

                _num_nodes = ceil(self.pooling_ratio * _num_nodes)

            elif _model_str == 'GMPool_I':

                pools.append(
                    PMA(_input_dim, self.num_heads, _num_nodes, ln=self.ln, cluster=self.cluster, mab_conv=None)
                )

                _num_nodes = ceil(self.pooling_ratio * _num_nodes)

            elif _model_str == 'SelfAtt':

                pools.append(
                    SAB(_input_dim, _output_dim, self.num_heads, ln=self.ln, cluster=self.cluster)
                )

                _input_dim = _output_dim
                _output_dim = _output_dim

            else:

                raise ValueError("Model Name in Model String <{}> is Unknown".format(_model_str))

        pools.append(nn.Linear(_input_dim, self.nhid))

        return pools

class GraphMultisetTransformer_for_OGB(GraphMultisetTransformer):

    def __init__(self, args):

        super(GraphMultisetTransformer_for_OGB, self).__init__(args)

        self.atom_encoder = AtomEncoder(self.nhid)
        self.convs = self.get_convs()

    def forward(self, data):

        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

        x = self.atom_encoder(x)

        # For Graph Convolution Network
        xs = []

        for _ in range(self.args.num_convs):

            x = F.relu(self.convs[_](x, edge_index, edge_attr))
            xs.append(x)

        # For jumping knowledge scheme
        x = torch.cat(xs, dim=1)

        # For Graph Multiset Transformer
        for _index, _model_str in enumerate(self.model_sequence):

            if _index == 0:

                batch_x, mask = to_dense_batch(x, batch)

                extended_attention_mask = mask.unsqueeze(1)
                extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)
                extended_attention_mask = (1.0 - extended_attention_mask) * -1e9

            if _model_str == 'GMPool_G':

                batch_x = self.pools[_index](batch_x, attention_mask=extended_attention_mask, graph=(x, edge_index, batch))

            else:

                batch_x = self.pools[_index](batch_x, attention_mask=extended_attention_mask)

            extended_attention_mask = None

        batch_x = self.pools[len(self.model_sequence)](batch_x)
        x = batch_x.squeeze(1)

        # For Classification
        x = self.classifier(x)

        return x

    def get_convs(self):

        convs = nn.ModuleList()

        for _ in range(self.args.num_convs):

            if self.args.conv == 'GCN':

                conv = GCNConv_for_OGB(self.nhid)

            elif self.args.conv == 'GIN':

                conv = GINConv_for_OGB(self.nhid)

            convs.append(conv)

        return convs




class NNDecoder(torch.nn.Module):
    def __init__(self, num_tasks, emb_dim, graph_pooling = "mean"):
        super().__init__()
        self.linear = torch.nn.Linear(emb_dim, num_tasks)

    def forward(self, node_rep):
      ## (~2-3 lines of code)
      output = self.linear(node_rep[:,0,:])
      return output

# baseline GCNConv comparison
def conv_OGB(dataset_name):

  path = '\tmp\ogb'
  # dataset = MoleculeNet(path, dataset_name).shuffle()
  ### Importing OGB dataset


  dataset = PygGraphPropPredDataset(name=dataset_name)
  evaluator = Evaluator(dataset_name)
  clf_criterion = torch.nn.BCEWithLogitsLoss()

  num_classes = dataset.num_classes
  num_features = dataset.num_features
  num_tasks = dataset.num_tasks
  hidden_channels = 32
  task_type='classification'
  # print(num_classes)

  def collate(data_list):
    batch = torch_geometric.data.Batch.from_data_list(data_list)
    return batch

  total_size = len(dataset)
  train_size = int(0.8 * total_size)
  test_size = (total_size - train_size) // 2
  val_size = total_size - train_size - test_size

  # Use random_split to split the dataset into train, test, and validation sets
  train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])

  train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=collate)
  val_loader = DataLoader(val_dataset, batch_size=128, collate_fn=collate)
  test_loader = DataLoader(test_dataset, batch_size=128, collate_fn=collate)

  class NN(torch.nn.Module):
      def __init__(self):
          super().__init__()

          self.conv1 = GCNConv(in_channels=num_features, out_channels=hidden_channels, normalize=True)
          # was hidden, num_tasks
          self.conv2 = GCNConv(in_channels=hidden_channels, out_channels=hidden_channels, normalize=True)
          self.relu = torch.nn.ReLU()

          # was 32, 16
          self.lin1 = Linear(hidden_channels, num_tasks)
          self.lin2 = Linear(16, num_tasks)

      def forward(self, x0, edge_index, batch):
          x = self.conv1(x0, edge_index)
          x = self.relu(x)
          x = self.conv2(x, edge_index)
          x = self.lin1(x)
          # global mean pool instead of GMT, along with other adjustments
          output = global_mean_pool(x, batch)

          return output


  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = NN().to(device)
  decoder = NNDecoder(emb_dim = 32, num_tasks = dataset.num_tasks).to(device)
  print(dataset.num_tasks)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

  def train(loader):
      model.train()

      total_loss = 0

      epoch_iter = tqdm(loader, ncols=0)
      for step, batch in enumerate(epoch_iter):
          batch = batch.to(device)
          optimizer.zero_grad()
          out = model(batch.x.float(), batch.edge_index, batch.batch)
          is_labeled = ~torch.isnan(batch.y) # Mask NaNs
          loss = clf_criterion(out.float()[is_labeled], batch.y.float()[is_labeled])
          loss.backward()
          # prevent exploding gradients if needed, uncomment
          # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
          total_loss += batch.num_graphs * float(loss)
          optimizer.step()
      print('loss returned:', total_loss / len(train_dataset))
      return total_loss / len(train_dataset)


  @torch.no_grad()
  def test(loader):

      model.eval()

      y_true = []
      y_pred = []

      total_same = 0
      epoch_iter = tqdm(loader, ncols=0)
      for step, batch in enumerate(epoch_iter):
          batch = batch.to(device)
          out = model(batch.x.float(), batch.edge_index, batch.batch)
          y_true.append(batch.y.view(out.shape).detach().cpu())
          y_pred.append(out.detach().cpu())
      y_true = torch.cat(y_true, dim = 0).numpy()
      y_pred = torch.cat(y_pred, dim = 0).numpy()
      input_dict = {"y_true": y_true, "y_pred": y_pred}

      return evaluator.eval(input_dict)


  times = []
  training_losses = []
  validation_metrics = []
  test_metrics = []
  for epoch in range(1, 201):
      start = time.time()
      train_loss = train(train_loader)
      val_acc = test(val_loader)
      test_acc = test(test_loader)
      training_losses.append(train_loss)
      validation_metrics.append(val_acc[dataset.eval_metric])
      test_metrics.append(test_acc[dataset.eval_metric])
      print(f'Epoch: {epoch:03d}, Loss: {train_loss:.4f}, '
            f'Val Acc: {val_acc}, Test Acc: {test_acc}')
      times.append(time.time() - start)
  best_val_epoch = np.argmax(np.array(validation_metrics))
  print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")
  plot_training_metrics(training_losses, validation_metrics, test_metrics, times)
  print('Best Validation Metric: {}'.format(validation_metrics[best_val_epoch]))
  print('Test Metric: {}'.format(test_metrics[best_val_epoch]))

conv_OGB('ogbg-moltoxcast')
conv_OGB('ogbg-molclintox')

# FPGNN

# required functions for our model

def remove_non_alphanumeric(input_string):
    # Use a regular expression to match only letters and numbers
    alphanumeric_pattern = re.compile(r'[\W_]+')

    # Replace non-alphanumeric characters with an empty string
    result_string = alphanumeric_pattern.sub('', input_string)

    return result_string

atom_type_max = 100
atom_f_dim = 133
atom_features_define = {
    'atom_symbol': list(range(atom_type_max)),
    'degree': [0, 1, 2, 3, 4, 5],
    'formal_charge': [-1, -2, 1, 2, 0],
    'charity_type': [0, 1, 2, 3],
    'hydrogen': [0, 1, 2, 3, 4],
    'hybridization': [
        Chem.rdchem.HybridizationType.SP,
        Chem.rdchem.HybridizationType.SP2,
        Chem.rdchem.HybridizationType.SP3,
        Chem.rdchem.HybridizationType.SP3D,
        Chem.rdchem.HybridizationType.SP3D2
    ],}

smile_changed = {}

def get_atom_features_dim():
    return atom_f_dim

def onek_encoding_unk(key,length):
    encoding = [0] * (len(length) + 1)
    index = length.index(key) if key in length else -1
    encoding[index] = 1

    return encoding

def get_atom_feature(atom):
    feature = onek_encoding_unk(atom.GetAtomicNum() - 1, atom_features_define['atom_symbol']) + \
           onek_encoding_unk(atom.GetTotalDegree(), atom_features_define['degree']) + \
           onek_encoding_unk(atom.GetFormalCharge(), atom_features_define['formal_charge']) + \
           onek_encoding_unk(int(atom.GetChiralTag()), atom_features_define['charity_type']) + \
           onek_encoding_unk(int(atom.GetTotalNumHs()), atom_features_define['hydrogen']) + \
           onek_encoding_unk(int(atom.GetHybridization()), atom_features_define['hybridization']) + \
           [1 if atom.GetIsAromatic() else 0] + \
           [atom.GetMass() * 0.01]
    return feature

class GraphOne:
    def __init__(self,smile,args):
        self.smile = smile
        self.atom_feature = []

        mol = Chem.MolFromSmiles(self.smile)
        # condition is added by me
        if mol is None:
          # print('baddddddd')
          smile = 'N#CCC1=CC=CC=C1'
          self.smile = smile
          mol = Chem.MolFromSmiles(self.smile)
        else:
          # print('good')
          blood = 7
        self.atom_num = mol.GetNumAtoms()

        for i, atom in enumerate(mol.GetAtoms()):
            self.atom_feature.append(get_atom_feature(atom))
        self.atom_feature = [self.atom_feature[i] for i in range(self.atom_num)]

class GraphBatch:
    def __init__(self,graphs,args):
        smile_list = []
        for graph in graphs:
            smile_list.append(graph.smile)
        self.smile_list = smile_list
        self.smile_num = len(self.smile_list)
        self.atom_feature_dim = get_atom_features_dim()
        self.atom_no = 1
        self.atom_index = []

        atom_feature = [[0]*self.atom_feature_dim]
        for graph in graphs:
            atom_feature.extend(graph.atom_feature)
            self.atom_index.append((self.atom_no,graph.atom_num))
            self.atom_no += graph.atom_num

        self.atom_feature = torch.FloatTensor(atom_feature)

    def get_feature(self):
        return self.atom_feature,self.atom_index

def create_graph(smile,args):
    graphs = []
    for one in smile:
        if one in smile_changed:
            graph = smile_changed[one]
        else:
            graph = GraphOne(one, args)
            smile_changed[one] = graph
        graphs.append(graph)
    return GraphBatch(graphs,args)

# our model is shown below

atts_out = []

class FPN(nn.Module):
    def __init__(self,args):
        super(FPN, self).__init__()
        self.fp_2_dim=args.fp_2_dim
        self.dropout_fpn = args.dropout
        self.cuda = args.cuda
        self.hidden_dim = args.hidden_size
        self.args = args
        if hasattr(args,'fp_type'):
            self.fp_type = args.fp_type
        else:
            self.fp_type = 'mixed'

        if self.fp_type == 'mixed':
            self.fp_dim = 1489
        else:
            self.fp_dim = 1024

        if hasattr(args,'fp_changebit'):
            self.fp_changebit = args.fp_changebit
        else:
            self.fp_changebit = None

        self.fc1=nn.Linear(self.fp_dim, self.fp_2_dim)
        self.act_func = nn.ReLU()
        self.fc2 = nn.Linear(self.fp_2_dim, self.hidden_dim)
        self.dropout = nn.Dropout(p=self.dropout_fpn)

    def forward(self, smile):
        fp_list=[]
        for i, one in enumerate(smile):
            fp=[]
            mol = Chem.MolFromSmiles(one)

            if self.fp_type == 'mixed':
                fp_maccs = AllChem.GetMACCSKeysFingerprint(mol)
                fp_phaErGfp = AllChem.GetErGFingerprint(mol,fuzzIncrement=0.3,maxPath=21,minPath=1)
                # fp_pubcfp = GetPubChemFPs(mol)
                fp.extend(fp_maccs)
                fp.extend(fp_phaErGfp)
                # fp.extend(fp_pubcfp)
            else:
                fp_morgan = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
                fp.extend(fp_morgan)
            fp_list.append(fp)

        if self.fp_changebit is not None and self.fp_changebit != 0:
            fp_list = np.array(fp_list)
            fp_list[:,self.fp_changebit-1] = np.ones(fp_list[:,self.fp_changebit-1].shape)
            fp_list.tolist()

        fp_list = torch.Tensor(fp_list)

        if self.cuda:
            fp_list = fp_list.cuda()
        fpn_out = self.fc1(fp_list)
        fpn_out = self.dropout(fpn_out)
        fpn_out = self.act_func(fpn_out)
        fpn_out = self.fc2(fpn_out)
        return fpn_out

class GATLayer(nn.Module):

    def __init__(self, in_features, out_features, dropout_gnn, alpha, inter_graph, concat=True):
        super(GATLayer, self).__init__()
        self.dropout_gnn= dropout_gnn
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat
        self.dropout = nn.Dropout(p=self.dropout_gnn)
        self.inter_graph = inter_graph

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)
        if self.inter_graph is not None:
            self.atts_out = []

    def forward(self,mole_out,adj):
        atom_feature = torch.mm(mole_out, self.W)
        N = atom_feature.size()[0]

        atom_trans = torch.cat([atom_feature.repeat(1, N).view(N * N, -1), atom_feature.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e = self.leakyrelu(torch.matmul(atom_trans, self.a).squeeze(2))

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)

        if self.inter_graph is not None:
            att_out = attention
            if att_out.is_cuda:
                att_out = att_out.cpu()
            att_out = np.array(att_out)
            att_out[att_out<-10000] = 0
            att_out = att_out.tolist()
            atts_out.append(att_out)

        attention = nn.functional.softmax(attention, dim=1)
        attention = self.dropout(attention)
        output = torch.matmul(attention, atom_feature)

        if self.concat:
            return nn.functional.elu(output)
        else:
            return output


class GATOne(nn.Module):
    def __init__(self,args):
        super(GATOne, self).__init__()
        self.nfeat = get_atom_features_dim()
        self.nhid = args.nhid
        self.dropout_gnn = args.dropout_gat
        self.atom_dim = args.hidden_size
        self.alpha = 0.2
        self.nheads = args.nheads
        self.args = args
        self.dropout = nn.Dropout(p=self.dropout_gnn)

        if hasattr(args,'inter_graph'):
            self.inter_graph = args.inter_graph
        else:
            self.inter_graph = None

        self.attentions = [GATLayer(self.nfeat, self.nhid, dropout_gnn=self.dropout_gnn, alpha=self.alpha, inter_graph=self.inter_graph, concat=True) for _ in range(self.nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = GATLayer(self.nhid * self.nheads, self.atom_dim, dropout_gnn=self.dropout_gnn, alpha=self.alpha, inter_graph=self.inter_graph, concat=False)

    def forward(self,mole_out,adj):
        mole_out = self.dropout(mole_out)
        mole_out = torch.cat([att(mole_out, adj) for att in self.attentions], dim=1)
        mole_out = self.dropout(mole_out)
        mole_out = nn.functional.elu(self.out_att(mole_out, adj))
        return nn.functional.log_softmax(mole_out, dim=1)

class GATEncoder(nn.Module):
    def __init__(self,args):
        super(GATEncoder,self).__init__()
        self.cuda = args.cuda
        self.args = args
        self.encoder = GATOne(self.args)

    def forward(self,mols,smiles):
        atom_feature, atom_index = mols.get_feature()
        if self.cuda:
            atom_feature = atom_feature.cuda()

        gat_outs=[]

        for i,one in enumerate(smiles):
            adj = []
            mol = Chem.MolFromSmiles(one)
            # added by me
            if mol is None:
              one = 'N#CCC1=CC=CC=C1'
              mol = Chem.MolFromSmiles(one)
            adj = Chem.rdmolops.GetAdjacencyMatrix(mol)
            # print('broooooooo')
            adj = adj/1
            adj = torch.from_numpy(adj)
            if self.cuda:
                adj = adj.cuda()

            atom_start, atom_size = atom_index[i]
            one_feature = atom_feature[atom_start:atom_start+atom_size]

            gat_atoms_out = self.encoder(one_feature,adj)
            gat_out = gat_atoms_out.sum(dim=0)/atom_size
            gat_outs.append(gat_out)
        gat_outs = torch.stack(gat_outs, dim=0)
        return gat_outs

class GAT(nn.Module):
    def __init__(self,args):
        super(GAT,self).__init__()
        self.args = args
        self.encoder = GATEncoder(self.args)

    def forward(self,smile):
        mol = create_graph(smile, self.args)
        gat_out = self.encoder.forward(mol,smile)

        return gat_out

class FpgnnModel(nn.Module):
    def __init__(self,is_classif,gat_scale,cuda,dropout_fpn):
        super(FpgnnModel, self).__init__()
        self.gat_scale = gat_scale
        self.is_classif = is_classif
        self.cuda = cuda
        self.dropout_fpn = dropout_fpn
        if self.is_classif:
            self.sigmoid = nn.Sigmoid()

    def create_gat(self,args):
        self.encoder3 = GAT(args)

    def create_fpn(self,args):
        self.encoder2 = FPN(args)

    def create_scale(self,args):
        linear_dim = int(args.hidden_size)
        if self.gat_scale == 1:
            self.fc_gat = nn.Linear(linear_dim,linear_dim)
        elif self.gat_scale == 0:
            self.fc_fpn = nn.Linear(linear_dim,linear_dim)
        else:
            self.gat_dim = int((linear_dim*2*self.gat_scale)//1)
            self.fc_gat = nn.Linear(linear_dim,self.gat_dim)
            self.fc_fpn = nn.Linear(linear_dim,linear_dim*2-self.gat_dim)
        self.act_func = nn.ReLU()

    def create_ffn(self,args):
        linear_dim = args.hidden_size
        if self.gat_scale == 1:
            self.ffn = nn.Sequential(
                                     nn.Dropout(self.dropout_fpn),
                                     nn.Linear(in_features=linear_dim, out_features=linear_dim, bias=True),
                                     nn.ReLU(),
                                     nn.Dropout(self.dropout_fpn),
                                     nn.Linear(in_features=linear_dim, out_features=args.task_num, bias=True)
                                     )
        elif self.gat_scale == 0:
            self.ffn = nn.Sequential(
                                     nn.Dropout(self.dropout_fpn),
                                     nn.Linear(in_features=linear_dim, out_features=linear_dim, bias=True),
                                     nn.ReLU(),
                                     nn.Dropout(self.dropout_fpn),
                                     nn.Linear(in_features=linear_dim, out_features=args.task_num, bias=True)
                                     )

        else:
            self.ffn = nn.Sequential(
                                     nn.Dropout(self.dropout_fpn),
                                     nn.Linear(in_features=linear_dim*2, out_features=linear_dim, bias=True),
                                     nn.ReLU(),
                                     nn.Dropout(self.dropout_fpn),
                                     nn.Linear(in_features=linear_dim, out_features=args.task_num, bias=True)
                                     )

    def forward(self,input):
        if self.gat_scale == 1:
            output = self.encoder3(input)
        elif self.gat_scale == 0:
            output = self.encoder2(input)
        else:
            gat_out = self.encoder3(input)
            fpn_out = self.encoder2(input)
            gat_out = self.fc_gat(gat_out)
            gat_out = self.act_func(gat_out)

            fpn_out = self.fc_fpn(fpn_out)
            fpn_out = self.act_func(fpn_out)

            output = torch.cat([gat_out,fpn_out],axis=1)
        output = self.ffn(output)

        if self.is_classif and not self.training:
            output = self.sigmoid(output)

        return output

def get_atts_out():
    return atts_out

def FPGNN(args):
    if args.dataset_type == 'classification':
        is_classif = 1
    else:
        is_classif = 0
    model = FpgnnModel(is_classif,args.gat_scale,args.cuda,args.dropout)
    if args.gat_scale == 1:
        model.create_gat(args)
        model.create_ffn(args)
    elif args.gat_scale == 0:
        model.create_fpn(args)
        model.create_ffn(args)
    else:
        model.create_gat(args)
        model.create_fpn(args)
        model.create_scale(args)
        model.create_ffn(args)

    for param in model.parameters():
        if param.dim() == 1:
            nn.init.constant_(param, 0)
        else:
            nn.init.xavier_normal_(param)

    return model



def run_fpgnn(dataset_name, task_name):


  dataset = MoleculeNet('\tmp\mol', dataset_name).shuffle()
  ### Importing OGB dataset

  # dataset2= = PygGraphPropPredDataset(name=task_name)
  evaluator = Evaluator(task_name)
  # Replace YourDataset with your actual dataset class
  # from your_module import YourDataset

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  args = Namespace(dataset_type="classification", gat_scale=1, cuda=False, dropout=0.25, nhid=128, dropout_gat=0.25, hidden_size=128, nheads=4,
                    task_num=617)
  model = FPGNN(args).to(device)

  optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
  batch_size=128
  train_size = int(0.8 * len(dataset))
  val_size = (len(dataset) - train_size) // 2
  test_size = len(dataset) - train_size - val_size

  # Split the dataset into train, validation, and test sets
  train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

  def collate(data_list):
      """
      Custom collate function to handle torch_geometric.data.data.Data instances.
      Converts a list of Data instances to a Batch.
      """
      return Batch.from_data_list(data_list)


  # Create data loaders with the custom collate function
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)

  clf_criterion = torch.nn.BCEWithLogitsLoss(torch.tensor(10))
  reg_criterion = torch.nn.MSELoss()




  def train(model, train_loader, criterion, optimizer, device):
      model.train()
      total_loss = 0.0

      epoch_iter = tqdm(train_loader, ncols=0)
      for step, data in enumerate(epoch_iter):
          data = data.to(device)
          smiles, labels = data.smiles, data.y

          optimizer.zero_grad()

          # Forward pass
          output = model(smiles)

          # Compute the loss
          is_labeled = ~torch.isnan(data.y) # Mask NaNs
          loss = criterion(output[is_labeled], data.y[is_labeled])

          # Backward pass and optimization
          loss.backward()
          # prevent exploding gradients
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
          optimizer.step()

          total_loss += loss.item()

      return total_loss / len(train_loader)

  @torch.no_grad()
  def test(model, test_loader, criterion, device):

      model.eval()

      y_true = []
      y_pred = []

      total_correct = 0
      epoch_iter = tqdm(test_loader, ncols=0)
      for step, batch in enumerate(epoch_iter):
          batch = batch.to(device)
          smiles, labels = batch.smiles, batch.y

          out = model(batch.smiles)
          output = out.cpu().numpy()
          print(out)
          print(batch.y)
          y_true.append(batch.y.view(out.shape).detach().cpu())
          y_pred.append(out.detach().cpu())
          total_correct += int((torch.argmax(out.to(float), dim=1) == torch.argmax(batch.y.to(float), dim=1)).sum())
      y_true = torch.cat(y_true, dim = 0).numpy()
      y_pred = torch.cat(y_pred, dim = 0).numpy()
      input_dict = {"y_true": y_true, "y_pred": y_pred}
      return evaluator.eval(input_dict)

  times = []
  training_losses = []
  validation_metrics = []
  test_metrics = []
  for epoch in range(1, 6):
      start = time.time()
      train_loss = train(model, train_loader, clf_criterion, optimizer, device)
      val_acc = test(model, val_loader, clf_criterion, device)
      test_acc = test(model, test_loader, clf_criterion, device)
      training_losses.append(train_loss)
      validation_metrics.append(val_acc)
      test_metrics.append(test_acc)
      print(f'Epoch: {epoch:03d}, Loss: {train_loss:.4f}, '
            f'Val Acc: {val_acc}, Test Acc: {test_acc}')
      times.append(time.time() - start)

run_fpgnn('ToxCast', 'ogbg-moltoxcast')

def run_fpgnn(dataset_name, task_name):

  dataset = MoleculeNet('\tmp\mol', dataset_name).shuffle()
  ### Importing OGB dataset
  evaluator = Evaluator(task_name)

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  args = Namespace(dataset_type="classification", gat_scale=1, cuda=False, dropout=0.25, nhid=128, dropout_gat=0.25, hidden_size=128, nheads=4,
                    task_num=2)
  model = FPGNN(args).to(device)
  # print(model)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

  batch_size=128
  train_size = int(0.8 * len(dataset))
  val_size = (len(dataset) - train_size) // 2
  test_size = len(dataset) - train_size - val_size
  # Split the dataset into train, validation, and test sets
  train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

  def collate(data_list):
      """
      Custom collate function to handle torch_geometric.data.data.Data instances.
      Converts a list of Data instances to a Batch.
      """
      return Batch.from_data_list(data_list)


  # Create data loaders with the custom collate function
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)

  clf_criterion = torch.nn.BCEWithLogitsLoss(torch.tensor(10))
  reg_criterion = torch.nn.MSELoss()




  def train(model, train_loader, criterion, optimizer, device):
      model.train()
      total_loss = 0.0

      epoch_iter = tqdm(train_loader, ncols=0)
      for step, data in enumerate(epoch_iter):
          data = data.to(device)
          smiles, labels = data.smiles, data.y

          optimizer.zero_grad()

          # Forward pass
          output = model(smiles)

          # Compute the loss
          is_labeled = ~torch.isnan(data.y) # Mask NaNs
          loss = criterion(output[is_labeled], data.y[is_labeled])

          # Backward pass and optimization
          loss.backward()
          # prevent exploding gradients
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
          optimizer.step()

          total_loss += loss.item()

      return total_loss / len(train_loader)

  @torch.no_grad()
  def test(model, test_loader, criterion, device):

      model.eval()

      y_true = []
      y_pred = []

      total_correct = 0
      epoch_iter = tqdm(test_loader, ncols=0)
      for step, batch in enumerate(epoch_iter):
          batch = batch.to(device)
          smiles, labels = batch.smiles, batch.y

          out = model(batch.smiles)
          output = out.cpu().numpy()

          y_true.append(batch.y.view(out.shape).detach().cpu())
          y_pred.append(out.detach().cpu())

          total_correct += int((torch.argmax(out.to(float), dim=1) == torch.argmax(batch.y.to(float), dim=1)).sum())
      y_true = torch.cat(y_true, dim = 0).numpy()
      y_pred = torch.cat(y_pred, dim = 0).numpy()
      input_dict = {"y_true": y_true, "y_pred": y_pred}
      return evaluator.eval(input_dict)

  times = []
  training_losses = []
  validation_metrics = []
  test_metrics = []
  for epoch in range(1, 6):
      start = time.time()
      train_loss = train(model, train_loader, clf_criterion, optimizer, device)
      val_acc = test(model, val_loader, clf_criterion, device)
      test_acc = test(model, test_loader, clf_criterion, device)
      training_losses.append(train_loss)
      validation_metrics.append(val_acc)
      test_metrics.append(test_acc)
      print(f'Epoch: {epoch:03d}, Loss: {train_loss:.4f}, '
            f'Val Acc: {val_acc}, Test Acc: {test_acc}')
      times.append(time.time() - start)

run_fpgnn('ClinTox', 'ogbg-molclintox')

dataset_dd = TUDataset('\tmp\dd', 'DD')
dataset_prot = TUDataset('\tmp\prot', 'PROTEINS')
dataset_cast = MoleculeNet('\tmp\cast', 'ToxCast')
dataset_clin = MoleculeNet('\tmp\clin', 'ClinTox')

dataset_list = [dataset_dd, dataset_prot, dataset_cast, dataset_clin]

data_dd = torch_geometric.data.Data(x=dataset_dd[0].x, edge_index=dataset_dd[0].edge_index)
g_dd = torch_geometric.utils.to_networkx(data_dd, to_undirected=True)
nx.draw(g_dd, node_size=50)


data_prot = torch_geometric.data.Data(x=dataset_prot[0].x, edge_index=dataset_prot[0].edge_index)
g_prot = torch_geometric.utils.to_networkx(data_prot, to_undirected=True)
nx.draw(g_prot, node_size=30)


data_cast = torch_geometric.data.Data(x=dataset_cast[0].x, edge_index=dataset_cast[0].edge_index)
g_cast = torch_geometric.utils.to_networkx(data_cast, to_undirected=True)
nx.draw(g_cast, node_size=50)

data_clin = torch_geometric.data.Data(x=dataset_clin[0].x, edge_index=dataset_clin[0].edge_index)
g_clin = torch_geometric.utils.to_networkx(data_clin, to_undirected=True)
nx.draw(g_clin, node_size=50)



def calculate_centrality(data, centrality_measure):
    # Convert the PyG data to a NetworkX graph
    G = torch_geometric.utils.to_networkx(data, to_undirected=True)

    # Calculate centrality measure
    if centrality_measure == 'degree':
        centrality_values = nx.degree_centrality(G).values()
    elif centrality_measure == 'betweenness':
        centrality_values = nx.betweenness_centrality(G).values()
    elif centrality_measure == 'closeness':
        centrality_values = nx.closeness_centrality(G).values()
    else:
        raise ValueError("Invalid centrality measure")

    return list(centrality_values)

# Example usage:

dataset_list = [dataset_dd, dataset_prot, dataset_cast, dataset_clin]
centrality_measure = 'degree'  # Change to 'betweenness' or 'closeness' as needed

for i, dataset in enumerate(dataset_list):
    all_centrality_values = []
    for data in dataset:
        centrality_values = calculate_centrality(data, centrality_measure)
        all_centrality_values.extend(centrality_values)

    # Plot histogram
    plt.hist(all_centrality_values, bins=20, alpha=0.5, label=f'Dataset {i + 1}')

plt.title(f'Centrality Distribution ({centrality_measure})')
plt.xlabel('Centrality Value')
plt.ylabel('Frequency')
plt.legend(labels=['D&D', 'PROTEINS', 'ToxCast', 'ClinTox'])
plt.show()

centrality_measure = 'betweenness'  # Change to 'betweenness' or 'closeness' as needed

for i, dataset in enumerate(dataset_list):
    all_centrality_values = []
    for data in dataset:
        centrality_values = calculate_centrality(data, centrality_measure)
        all_centrality_values.extend(centrality_values)

    # Plot histogram
    plt.hist(all_centrality_values, bins=20, alpha=0.5, label=f'Dataset {i + 1}')

plt.title(f'Centrality Distribution ({centrality_measure})')
plt.xlabel('Centrality Value')
plt.ylabel('Frequency')
plt.legend(labels=['D&D', 'PROTEINS', 'ToxCast', 'ClinTox'])
plt.show()

centrality_measure = 'closeness'  # Change to 'betweenness' or 'closeness' as needed

for i, dataset in enumerate(dataset_list):
    all_centrality_values = []
    for data in dataset:
        centrality_values = calculate_centrality(data, centrality_measure)
        all_centrality_values.extend(centrality_values)

    # Plot histogram
    plt.hist(all_centrality_values, bins=20, alpha=0.5, label=f'Dataset {i + 1}')

plt.title(f'Centrality Distribution ({centrality_measure})')
plt.xlabel('Centrality Value')
plt.ylabel('Frequency')
plt.legend(labels=['D&D', 'PROTEINS', 'ToxCast', 'ClinTox'])
plt.show()

for dataset in dataset_list:
  print(dataset)
  print('Number of classes:', dataset.num_classes)

  # Use DataLoader to iterate through the dataset
  data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

  # Initialize two variables to store the total number of nodes and edges
  total_nodes = 0
  total_edges = 0

  # Iterate through the dataset
  graph_sizes = []
  for data in data_loader:
      num_nodes = data.num_nodes
      total_nodes += num_nodes
      # deal with outliers
      if num_nodes != 0:
        graph_sizes.append(num_nodes)
      num_edges = data.edge_index.size(1)
      total_edges += num_edges

  # Calculate the max and min number of nodes
  max_nodes = max(graph_sizes)
  min_nodes = min(graph_sizes)

  # Calculate the mean number of nodes and edges
  mean_nodes = total_nodes / len(dataset)
  mean_edges = total_edges / len(dataset)

  print("Max number of nodes:", max_nodes)
  print("Min number of nodes:", min_nodes)
  print("Mean number of nodes:", mean_nodes)
  print("Mean number of edges:", mean_edges)
  print("Number node features:", dataset.num_features)
  print("NUmber edge features:", dataset.num_edge_features)



# outputs
pb = [0.7297297297297297, 0.6936936936936937, 0.6306306306306306]
db = [0.7203389830508474, 0.7203389830508474, 0.8135593220338984]
cab = [0.6550131347453804, 0.6489285967448071, 0.6307807797930923]
clb = [0.6024523124467178, 0.6520326747720365, 0.6441915397970086]
pa = [0.7143, 0.6875, 0.7054]
da = [0.635593220338983, 0.5932203389830508, 0.66429384723094]
caf = [0.65763476286, 0.5876234768, 0.64854958745]
clf = [0.58834628374, 0.5736284762387, 0.66642837468]

# Calculate mean and standard error
print(np.mean(pb))
print(np.std(pb, ddof=1) / np.sqrt(len(pb)))
print(np.mean(db))
print(np.std(db, ddof=1) / np.sqrt(len(db)))
print(np.mean(cab))
print(np.std(cab, ddof=1) / np.sqrt(len(cab)))
print(np.mean(clb))
print(np.std(clb, ddof=1) / np.sqrt(len(clb)))
print('----')
print(np.mean(pa))
print(np.std(pa, ddof=1) / np.sqrt(len(pa)))
print(np.mean(pb))
print(np.std(da, ddof=1) / np.sqrt(len(da)))
print('-----')
print(np.mean(caf))
print(np.std(caf, ddof=1) / np.sqrt(len(caf)))
print(np.mean(clf))
print(np.std(clf, ddof=1) / np.sqrt(len(clf)))
